In [ ]:
%load_ext autoreload
%autoreload 2 

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import pystorm
from pystorm.hal import HAL
from pystorm.hal.net_builder import NetBuilder
from pystorm.hal.run_control import RunControl

In [ ]:
# call the function
hal = HAL()
nb = NetBuilder(hal)

bad_syns, dbg = nb.determine_bad_syns()

In [ ]:
# this was basically my scratchpad to write the function, does basically the same thing
# compare to function output afterwards
plt.figure()
plt.title('pulse widths by position')
pw = dbg['pulse_widths']
plt.imshow(pw)

med = np.median(pw)
std = np.std(pw)
print(med, std)

plt.figure()
plt.title('pulse widths, lines are median +/- 1 std')
plt.hist(pw.flatten(), bins=20)
ax = plt.gca()
plt.axvline(med, color='r')
plt.axvline(med - std, color='r')
plt.axvline(med + std, color='r')

plt.figure()
plt.title('pulse widths, lines are attrition cutoffs')
plt.hist(pw.flatten(), bins=20)
ax = plt.gca()

pwflat = pw.flatten()
fmaxes = 1 / pwflat
order = np.argsort(fmaxes)
fmax_for_attritions = []
attritions = np.linspace(0, .5, 11)
for attrition in attritions:
    cutoff_idx = order[int(attrition * len(fmaxes))]
    val = fmaxes[cutoff_idx]
    plt.axvline(1 / val, color='r')
    print('fmax at attrition of ' + "{:0.2f}".format(attrition) + ' is ' + str(val))
    fmax_for_attritions.append(val)
    
    
plt.figure()
plt.title('fmax vs attrition')
plt.plot(attritions, fmax_for_attritions)
plt.xlabel('attrition')
plt.ylabel('fmax')



In [ ]:
# study correlation of high bias and slow pulse synapses
hb = dbg['high_bias']
plt.figure()
plt.title('high bias syns')
plt.imshow(hb)

bad_pw = fmaxes < fmax_for_attritions[1]
plt.figure()
plt.title('slow pulse syns')
plt.imshow(bad_pw.reshape(32, 32))

plt.figure()
plt.figure('both')
plt.imshow(bad_pw.reshape(32, 32) & hb)

redo_bad_syn = bad_pw.reshape(32, 32) | hb
print(np.sum(redo_bad_syn), "lost total neurons")

In [ ]:
# compare to output of call
plt.figure()
plt.title('XOR of function output and work in this notebook')
plt.imshow(bad_syns & redo_bad_syn | ~bad_syns & ~redo_bad_syn) # xor